In [ ]:
!pip install nltk
!pip install spacy
!pip install en_core_web_sm
!pip install numpy

In [ ]:
!python3 -m spacy download en_core_web_sm

In [1]:
import nltk
nltk.download('movie_reviews')
nltk.download("subjectivity")
nltk.download('punkt')
from nltk.corpus import movie_reviews
from sklearn.metrics import classification_report
import spacy
import en_core_web_sm

from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
nlp = en_core_web_sm.load()


mr = movie_reviews


from nltk.corpus import subjectivity
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer, VaderConstants

analyzer = SentimentIntensityAnalyzer()
doc = subjectivity.sents()

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/disi/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package subjectivity to
[nltk_data]     /home/disi/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!
[nltk_data] Downloading package punkt to /home/disi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/disi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
def lol2str(doc):
    # flatten & join
    return "".join([w for sent in doc for w in sent])

def score_sent(document, use_pos=False):
    pos = []
    neg = []
    obj = []
    for sent in document:
        if use_pos:
            tagged_sent = pos_tag(sent, tagset='universal')
        else:
            tagged_sent = [(w, None) for w in sent]

        for tok, tag in tagged_sent:
            ss = lesk(sent, tok, pos=pos2wn.get(tag, None))
            if ss:
                sense = swn.senti_synset(ss.name())
                pos.append(sense.pos_score())
                neg.append(sense.neg_score())
                obj.append(sense.obj_score())
    return pos, neg, obj

def subjectivity_sentence_level(document, analyzer):
    S = 0
    O = 0
    labels = ['S', 'O']
    for sentence in document:
        value = analyzer.polarity_scores(" ".join(sentence))
        if value["compound"] != 0:
            S+=1
        else:
            O+=1
    return labels[numpy.argmax(numpy.asarray([S, O]))]
def rm_objective_sentences(document, analyzer):
    new_doc = []
    for sentence in document:
        value = analyzer.polarity_scores(" ".join(sentence))
        if value["compound"] != 0:
            new_doc.append(" ".join(sentence))
    return new_doc
def separate_objective_sentences(document, analyzer):
    new_doc = []
    obj_doc = []
    for sentence in document:
        value = analyzer.polarity_scores(" ".join(sentence))
        if value["compound"] != 0:
            new_doc.append(" ".join(sentence))
        else:
            obj_doc.append(" ".join(sentence))
    return new_doc, obj_doc
def polarity_doc_level(document, analyzer):
    value = analyzer.polarity_scores(document)
    P = 0
    N = 0
    labels = ['P', 'N']
    if value["compound"] > 0:
        P += 1
    elif value["compound"] <= 0: # In this way we penalize the neg class
        N += 1
    return labels[numpy.argmax(numpy.asarray([P, N]))]

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

# Train and test with Stratified K Fold

skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
scores_clf = []
scores_subjectivity = []
new_doc = []

#corpus = [lol2str(d) for d in rev_neg] + [lol2str(d) for d in rev_pos]
#vectors = vectorizer.fit_transform(mr)

ref_sub, ref_ob = separate_objective_sentences(doc, analyzer)
corpus = [lol2str(d) for d in ref_ob] + [lol2str(d) for d in ref_sub]
ref = numpy.array([0] * len(ref_ob) + [1] * len(ref_sub))

for i, (train_index, test_index) in enumerate(skf.split(corpus, ref)):
    x_train, x_test = [corpus[indx] for indx in train_index], [corpus[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level

    test_x_split = [[sentence.split() for sentence in corpus] for corpus in x_test]
    #[item for sublist in nested_list for item in sublist]
  
    vectorizer = CountVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    #scores = cross_validate(clf, vectors, labels, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    new_doc.append(hyp)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    #hyp_sentence = [subjectivity_sentence_level(doc, analyzer) for doc in test_x_split]
    #scores_subjectivity.append(f1_score(y_test, hyp_sentence, average='macro'))
        
    
print('\033[1mSubjectivity: F1 classifier:\033[0m', round(sum(scores_clf)/len(scores_clf), 3))
print('\033[1mSubjectivity: Classification Report:\033[0m')
print(classification_report(y_test, hyp))
#print('F1 Subjectivity:',  round(sum(scores_subjectivity)/len(scores_subjectivity), 3))

In [ ]:
scores_clf = []


rev_neg = mr.paras(categories='neg')
rev_pos = mr.paras(categories='pos')

rev_neg_wo_objective = ["\n".join(rm_objective_sentences(doc, analyzer)) for doc in rev_neg]
rev_pos_wo_objective = ["\n".join(rm_objective_sentences(doc, analyzer)) for doc in rev_pos]

corpus_wo_objective = rev_neg_wo_objective + rev_pos_wo_objective
#corpus = [lol2str(d) for d in ref_ob] + [lol2str(d) for d in ref_sub]
ref = numpy.array([0] * len(rev_neg_wo_objective) + [1] * len(rev_pos_wo_objective))

for i, (train_index, test_index) in enumerate(skf.split(corpus_wo_objective, ref)):
    x_train, x_test = [corpus_wo_objective[indx] for indx in train_index], [corpus_wo_objective[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = CountVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    #scores = cross_validate(clf, vectors, labels, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    #hyp_sentence = [subjectivity_sentence_level(doc, analyzer) for doc in test_x_split]
    #scores_subjectivity.append(f1_score(y_test, hyp_sentence, average='macro'))
        
    
print('\033[1mF1 classifier Polarity w/o Obj:\033[0m', round(sum(scores_clf)/len(scores_clf), 3))
print('\033[1mPolarity w/o Obj: Classification Report:\033[0m')
print(classification_report(y_test, hyp))

In [ ]:
scores_clf = []


rev_neg = mr.paras(categories='neg')
rev_pos = mr.paras(categories='pos')
corpus = [lol2str(d) for d in rev_neg] + [lol2str(d) for d in rev_pos]
#vectors = vectorizer.fit_transform(mr)

ref = numpy.array([0] * len(rev_neg) + [1] * len(rev_pos))

for i, (train_index, test_index) in enumerate(skf.split(corpus, ref)):
    x_train, x_test = [corpus[indx] for indx in train_index], [corpus[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]

    vectorizer = CountVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)
    
    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    #scores = cross_validate(clf, vectors, ref, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))
    
    #hyp_polarity = [polarity_doc_level(doc, analyzer) for doc in x_test]
    #scores_polarity.append(f1_score(y_test, hyp_polarity, average='macro'))

    
    
print('\033[1mF1 classifier Polarity with Obj:\033[0m', round(sum(scores_clf)/len(scores_clf), 3))
print('\033[1mPolarity with Obj: Classification Report:\033[0m')
print(classification_report(y_test, hyp))

In [85]:
#from utils import *
import numpy
import os
import re

SMALL_POSITIVE_CONST = 1e-4

def load_data(path):
    '''
        input: path/to/data
        output: json
    '''
    dataset = []
    with open(path, 'r', encoding='utf-8') as f:
        #dataset = f.readlines()
    #with open(file_path, 'r') as file:
        for line in f:
            # Use regular expression to find the sentence before the symbol
            match = re.search(r'(.*?){}'.format(re.escape('####')), line)

            if match:
                sentence_before_symbol = match.group(1).strip()
                dataset.append(sentence_before_symbol)

    return dataset



def extract(doc):
    aspects = []
    for sent in doc.sents:
        target = None
        opinion = None
        for tok in sent:
            if tok.dep_ == 'nsubj' and tok.pos_ == 'NOUN':
                target = tok.text
            if tok.pos_ == 'ADJ':
                descr = ''
                for child in tok.children:
                    if child.pos_ != 'ADV':
                        continue
                    descr += child.text + ' '
                opinion = descr + tok.text
        if target:
            aspects.append((target, opinion))
    return aspects

def tag2ot(ote_tag_sequence):
    """
    transform ote tag sequence to a sequence of opinion target
    :param ote_tag_sequence: tag sequence for ote task
    :return:
    """
    n_tags = len(ote_tag_sequence)
    ot_sequence = []
    beg, end = -1, -1
    for i in range(n_tags):
        tag = ote_tag_sequence[i]
        if tag == 'S':
            ot_sequence.append((i, i))
        elif tag == 'B':
            beg = i
        elif tag == 'E':
            end = i
            if end > beg > -1:
                ot_sequence.append((beg, end))
                beg, end = -1, -1
    return ot_sequence

def evaluate_ote(gold_ot, pred_ot):
    """
    evaluate the model performce for the ote task
    :param gold_ot: gold standard ote tags
    :param pred_ot: predicted ote tags
    :return:
    """
    assert len(gold_ot) == len(pred_ot)
    n_samples = len(gold_ot)
    # number of true positive, gold standard, predicted opinion targets
    n_tp_ot, n_gold_ot, n_pred_ot = 0, 0, 0
    #for i in range(n_samples):
        #g_ot = gold_ot[i]
        #p_ot = pred_ot[i]
    g_ot_sequence, p_ot_sequence = tag2ot(ote_tag_sequence=gold_ot), tag2ot(ote_tag_sequence=pred_ot)
    # hit number
    n_hit_ot = match_ot(gold_ote_sequence=g_ot_sequence, pred_ote_sequence=p_ot_sequence)
    n_tp_ot += n_hit_ot
    n_gold_ot += len(g_ot_sequence)
    n_pred_ot += len(p_ot_sequence)
    # add 0.001 for smoothing
    # calculate precision, recall and f1 for ote task
    ot_precision = float(n_tp_ot) / float(n_pred_ot + SMALL_POSITIVE_CONST)
    ot_recall = float(n_tp_ot) / float(n_gold_ot + SMALL_POSITIVE_CONST)
    ot_f1 = 2 * ot_precision * ot_recall / (ot_precision + ot_recall + SMALL_POSITIVE_CONST)
    ote_scores = (ot_precision, ot_recall, ot_f1)
    return ote_scores

def tag2ts(ts_tag_sequence):
    """
    transform ts tag sequence to targeted sentiment
    :param ts_tag_sequence: tag sequence for ts task
    :return:
    """
    n_tags = len(ts_tag_sequence)
    ts_sequence, sentiments = [], []
    beg, end = -1, -1
    for i in range(n_tags):
        ts_tag = ts_tag_sequence[i]
        # current position and sentiment
        ts_tag = ts_tag.astype(str).tolist()
        eles = ts_tag.split('-')
        if len(eles) == 2:
            pos, sentiment = eles
        else:
            pos, sentiment = 'O', 'O'
        if sentiment != 'O':
            # current word is a subjective word
            sentiments.append(sentiment)
        if pos == 'S':
            # singleton
            ts_sequence.append((i, i, sentiment))
            sentiments = []
        elif pos == 'B':
            beg = i
        elif pos == 'E':
            end = i
            # schema1: only the consistent sentiment tags are accepted
            # that is, all of the sentiment tags are the same
            if end > beg > -1 and len(set(sentiments)) == 1:
                ts_sequence.append((beg, end, sentiment))
                sentiments = []
                beg, end = -1, -1
    return ts_sequence

def evaluate_ts(gold_ts, pred_ts):
    """
    evaluate the model performance for the ts task
    :param gold_ts: gold standard ts tags
    :param pred_ts: predicted ts tags
    :return:
    """
    assert len(gold_ts) == len(pred_ts)
    n_samples = len(gold_ts)
    # number of true postive, gold standard, predicted targeted sentiment
    n_tp_ts, n_gold_ts, n_pred_ts = numpy.zeros(3), numpy.zeros(3), numpy.zeros(3)
    ts_precision, ts_recall, ts_f1 = numpy.zeros(3), numpy.zeros(3), numpy.zeros(3)

    #for i in range(n_samples):
        #g_ts = gold_ts[i]
        #p_ts = pred_ts[i]
    g_ts_sequence, p_ts_sequence = tag2ts(ts_tag_sequence=gold_ts), tag2ts(ts_tag_sequence=pred_ts)
    hit_ts_count, gold_ts_count, pred_ts_count = match_ts(gold_ts_sequence=g_ts_sequence,
                                                          pred_ts_sequence=p_ts_sequence)

    n_tp_ts += hit_ts_count
    n_gold_ts += gold_ts_count
    n_pred_ts += pred_ts_count
        # calculate macro-average scores for ts task
    for i in range(3):
        n_ts = n_tp_ts[i]
        n_g_ts = n_gold_ts[i]
        n_p_ts = n_pred_ts[i]
        ts_precision[i] = float(n_ts) / float(n_p_ts + SMALL_POSITIVE_CONST)
        ts_recall[i] = float(n_ts) / float(n_g_ts + SMALL_POSITIVE_CONST)
        ts_f1[i] = 2 * ts_precision[i] * ts_recall[i] / (ts_precision[i] + ts_recall[i] + SMALL_POSITIVE_CONST)

    ts_macro_f1 = ts_f1.mean()

    # calculate micro-average scores for ts task
    n_tp_total = sum(n_tp_ts)
    # total sum of TP and FN
    n_g_total = sum(n_gold_ts)
    # total sum of TP and FP
    n_p_total = sum(n_pred_ts)

    ts_micro_p = float(n_tp_total) / (n_p_total + SMALL_POSITIVE_CONST)
    ts_micro_r = float(n_tp_total) / (n_g_total + SMALL_POSITIVE_CONST)
    ts_micro_f1 = 2 * ts_micro_p * ts_micro_r / (ts_micro_p + ts_micro_r + SMALL_POSITIVE_CONST)
    ts_scores = (ts_macro_f1, ts_micro_p, ts_micro_r, ts_micro_f1)
    return ts_scores


def evaluate(gold_ot, gold_ts, pred_ot, pred_ts):
    """
    evaluate the performance of the predictions
    :param gold_ot: gold standard opinion target tags
    :param gold_ts: gold standard targeted sentiment tags
    :param pred_ot: predicted opinion target tags
    :param pred_ts: predicted targeted sentiment tags
    :return: metric scores of ner and sa
    """
    assert len(gold_ot) == len(gold_ts) == len(pred_ot) == len(pred_ts)
    ote_scores = evaluate_ote(gold_ot=gold_ot, pred_ot=pred_ot)
    ts_scores = evaluate_ts(gold_ts=gold_ts, pred_ts=pred_ts)
    return ote_scores, ts_scores


def match_ot(gold_ote_sequence, pred_ote_sequence):
    """
    calculate the number of correctly predicted opinion target
    :param gold_ote_sequence: gold standard opinion target sequence
    :param pred_ote_sequence: predicted opinion target sequence
    :return: matched number
    """
    n_hit = 0
    for t in pred_ote_sequence:
        if t in gold_ote_sequence:
            n_hit += 1
    return n_hit


def match_ts(gold_ts_sequence, pred_ts_sequence):
    """
    calculate the number of correctly predicted targeted sentiment
    :param gold_ts_sequence: gold standard targeted sentiment sequence
    :param pred_ts_sequence: predicted targeted sentiment sequence
    :return:
    """
    # positive, negative and neutral
    tag2tagid = {'POS': 0, 'NEG': 1, 'NEU': 2}
    hit_count, gold_count, pred_count = numpy.zeros(3), numpy.zeros(3), numpy.zeros(3)
    for t in gold_ts_sequence:
        #print(t)
        ts_tag = t[2]
        tid = tag2tagid[ts_tag]
        gold_count[tid] += 1
    for t in pred_ts_sequence:
        ts_tag = t[2]
        tid = tag2tagid[ts_tag]
        if t in gold_ts_sequence:
            hit_count[tid] += 1
        pred_count[tid] += 1
    return hit_count, gold_count, pred_count

def polarity_aspect(aspects, analyzer):
    pos = 0
    neg = 0
    labels = ['P', 'N']
    rev_t_neg = []
    rev_t_pos = []
    rev_o_neg = []
    rev_o_pos = []
    
    for target, opinion in aspects:
        if opinion is None:
            opinion = 'Not Related'
        else:
            value = analyzer.polarity_scores(opinion)
            if value["compound"] > 0:
                pos += 1 
                rev_t_pos.append(target)
                rev_o_pos.append(opinion)
            elif value["compound"] < 0:
                neg += 1
                rev_t_neg.append(target)
                rev_o_neg.append(opinion)
    return rev_t_pos, rev_o_pos, rev_t_neg, rev_o_neg

def polarity_doc_level(document, analyzer):
    rev_neg = []
    rev_pos = []
    value = analyzer.polarity_scores(document)
    if value["compound"] > 0:
        rev_pos.append(sentence)
    elif value["compound"] <= 0: # In this way we penalize the neg class
        rev_neg.append(sentence) 
    return rev_neg, rev_pos

In [47]:
train_raw = load_data(os.path.join('dataset/laptop14_train.txt'))
test_raw = load_data(os.path.join('dataset/laptop14_test.txt'))
#print(nlp(' '.join(train_raw)))
aspects_train = extract(nlp(' '.join(train_raw)))

aspects_test = extract(nlp(' '.join(test_raw)))

In [91]:
scores_clf = []
hyp_targ = []
ref_targ = []
hyp_op = []
ref_op = []

rev_t_pos, rev_o_pos, rev_t_neg, rev_o_neg = polarity_aspect(aspects_train, analyzer)
corpus_trn = [lol2str(d) for d in rev_t_neg] + [lol2str(d) for d in rev_t_pos] + [lol2str(d) for d in rev_o_neg] + [lol2str(d) for d in rev_o_pos]
#vectors = vectorizer.fit_transform(mr)
ref_trn = numpy.array([0] * len(rev_t_neg) + [1] * len(rev_t_pos) + [2] * len(rev_o_neg) + [3] * len(rev_o_pos))

rev_t_pos2, rev_o_pos2, rev_t_neg2, rev_o_neg2 = polarity_aspect(aspects_test, analyzer)
corpus_tst = [lol2str(d) for d in rev_t_neg2] + [lol2str(d) for d in rev_t_pos2] + [lol2str(d) for d in rev_o_neg2] + [lol2str(d) for d in rev_o_pos2]
#vectors = vectorizer.fit_transform(mr)

ref_tst = numpy.array([0] * len(rev_t_neg2) + [1] * len(rev_t_pos2) + [2] * len(rev_o_neg2) + [3] * len(rev_o_pos2))

#for (train_index, test_index) in enumerate(skf.split(corpus, ref)):
#for i, (train_index, test_index) in enumerate(skf.split(corpus_trn, ref_trn), skf.split(corpus_tst, ref_tst)):
for i, ((train_index, train_index), (test_index, test_index)) in enumerate(zip(skf.split(corpus_trn, ref_trn), skf.split(corpus_tst, ref_tst))):

    x_train, x_test = [corpus_trn[idx] for idx in train_index], [corpus_tst[idx] for idx in test_index]
    y_train, y_test = [ref_trn[idx] for idx in train_index], [ref_tst[idx] for idx in test_index]

    # Needed for word and sentence level
    #test_x_split = [[sentence for sentence in doc] for doc in x_test]

    #x_train = ' '.join(x_train[0])
    #x_test = [' '.join(doc) for doc in x_test]
    #unequal_indices = numpy.where(x_train != y_train)

    # Increase elements in array1 at unequal indices
    #array1[unequal_indices] += 1
    vectorizer = CountVectorizer()
    vectorizer.fit(x_train)
    train_features = vectorizer.transform(x_train)
    test_features = vectorizer.transform(x_test)

    clf = MLPClassifier(random_state=1, max_iter=300).fit(train_features, y_train)
    #scores = cross_validate(clf, vectors, ref, cv=StratifiedKFold(n_splits=10) , scoring=['f1_micro'])
    hyp = clf.predict(test_features)
    scores_clf.append(f1_score(y_test, hyp, average='macro'))

    #hyp_polarity = [polarity_doc_level(doc, analyzer) for doc in asp_doc_test]
    #scores_polarity.append(f1_score(y_test, hyp_polarity, average='macro'))

    
print('\033[1mF1 classifier Polarity of aspect terms:\033[0m', round(sum(scores_clf)/len(scores_clf), 3))
print('\033[1mPolarity of Aspect terms: Classification Report\033[0m:')
print(classification_report(y_test, hyp))
print(len(hyp))
print(len(y_test))
for i in hyp:
    if i==1:
        hyp_targ.append(i)
        ref_targ.append(i)
    else:
        hyp_op.append(i)
        ref_op.append(i)
print('\033[1mSEMEVAL Polarity of aspect terms:\033[0m')
print(evaluate(y_test, y_test, hyp, hyp))
print(evaluate(ref_op, ref_targ, hyp_op, hyp_targ))

/home/disi/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/disi/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/disi/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/disi/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/disi/.local/lib/python3.8/site-packages/sklearn/neural

F1 classifier Polarity of aspect terms: 0.476
Polarity of Aspect terms: Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.54      0.88      0.67         8
           2       1.00      1.00      1.00         1
           3       1.00      0.25      0.40         8

    accuracy                           0.61        18
   macro avg       0.76      0.78      0.68        18
weighted avg       0.77      0.61      0.57        18

18
18
SEMEVAL Polarity of aspect terms:
((0.0, 0.0, 0.0), (0.0, 0.0, 0.0, 0.0))


/home/disi/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


AssertionError: 

In [ ]:
txt = """The setting of this hotel is really lovely. 
Rooms are spacious, spotlessly clean, with large and comfortable beds. 
We appreciated the courtesy touches, such as the fresh fruit on the table, always replaced.

However, our room, in the new building, came at 200 euros a night (admittedly it was a busy weekend), and at this price one begins of course to be more demanding and less forgiving :-) , so let me mention that I would expect a better shower, and that the parking and manoeuvrability situation is not the best (narrow passage + slight chaos when full).

But I want to conclude on a positive note: their breakfast is very good, with a sense of abundance, fresh juices, many jams and teas and cereals and nuts, proper just prepared scrambled eggs, (note for Anglo-Saxon visitors: it's mostly "European" style, so apart from the eggs don't expect many other hot/cooked items). And service (as well as reception) is excellent, with some people having more natural human warmth than others, but always courteous and helpful."""
doc = nlp(txt)
print(doc)
for sent in doc.sents:
    for tok in sent:
        if tok.dep_ == 'nsubj' and tok.pos_ == 'NOUN':
            print(tok.text)
aspects = extract(doc)
print(aspects)
#print(train_raw)
aspects = extract(nlp(' '.join(train_raw)))
print(aspects)

In [ ]:
print(nlp(' '.join(train_raw)))

In [37]:
scores_clf = []
scores_polarity = []
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
rev_neg = mr.paras(categories='neg')
rev_pos = mr.paras(categories='pos')
corpus = [lol2str(d) for d in rev_neg] + [lol2str(d) for d in rev_pos]
#vectors = vectorizer.fit_transform(mr)

ref = numpy.array([0] * len(rev_neg) + [1] * len(rev_pos))

for i, (train_index, test_index) in enumerate(skf.split(corpus, ref)):
    x_train, x_test = [corpus[indx] for indx in train_index], [corpus[indx] for indx in test_index]
    y_train, y_test = [ref[indx] for indx in train_index], [ref[indx] for indx in test_index]
    # Needed for word and sentence level
    test_x_split = [[sentence.split() for sentence in doc.splitlines()] for doc in x_test]
    vectorizer = CountVectorizer()
    vectorizer.fit(x_train)
    print(len(x_train))
    print(len(y_train))

1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
1800
